In [18]:
import sys
import json
sys.path.append('..')
from src.analysis.model_performances import *
import pandas as pd
EVERYTHING_POSTFIX = 'everything'

In [19]:
def to_df(results_all, model_name, answer_type, prompt_type, subs, domains):
    d_ramifications_pretty = {
        WITH_RAMIFICATIONS : 'R',
        WITHOUT_RAMIFICATIONS : 'No R'
    }
    d_answer_type_pretty = {
        TRUE_FALSE_ANSWER_TYPE : 'T/F',
        FREE_ANSWER_TYPE : 'Free'
    }
    
    index = []
    data = []
    for plan_length in PLAN_LENGTHS:
        for question_category in QUESTION_CATEGORIES + [ALL_QUESTION_CATEGORIES_KEY]:
            for ramifications in RAMIFICATION_TYPES:
                index.append((plan_length, question_category, d_ramifications_pretty[ramifications], d_answer_type_pretty[answer_type]))
                data_columns = {}
                for domain in domains:
                    instance = filter_single_selector(results_all, plan_length, question_category, ramifications, model_name, prompt_type, domain,
                          answer_type, subs)
                    if len(instance) == 0:
                        data_columns[domain] = None
                    elif not len(instance) == 1: 
                        raise ValueError(f'len(instance) == {len(instance)}')
                    else:
                        data_columns[domain] = instance[0][SK_RESULT]
                data.append(data_columns)
    return pd.DataFrame(data, index = index)

# def df_to_latex_table(df):
#         latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)
#         return latex_table[latex_table.find('\midrule'):].replace("bottomrule", 'crap').replace("\crap", '').replace("\end{tabular}", '')

# def assemble_table(results_all, answer_type, domain, score_key=None):
#     latex_table_all = ''
#     with open('latex_table_template/top') as f:
#         latex_table_all += f.read() + '\n'
#     latex_table_all += '\n'.join([df_to_latex_table(to_df(results_all, answer_type, plan_length, domain)) for plan_length in PLAN_LENGTHS])
#     with open('latex_table_template/bottom') as f:
#         latex_table_all += f.read()
#     
#     caption = f'{answer_type}, {score_key} scores for {domain}'.replace('_', ' ')
#     latex_table_all = latex_table_all.replace('REPLACE_CAPTION_KEY', caption)
#     
#     return latex_table_all


In [20]:
answer_type = TRUE_FALSE_ANSWER_TYPE #FREE_ANSWER#
score_key = F1_SCORE_KEY#'accuracy'
answer_type_ext = tf_answer_type(score_key = F1_SCORE_KEY)
stats_all = collect_stats_all(tf_answer_type(score_key = F1_SCORE_KEY))

100%|██████████| 10240/10240 [00:01<00:00, 5288.34it/s]


In [25]:
model_name = 'gemma-2b-it'
prompt_type = 'few_shot_1'
subs = WITHOUT_RANDOM_SUB
# domains= DOMAIN_NAMES + [ALL_DOMAINS_KEY]
domains = [TRANSPORTATION_DOMAIN_KEY, NON_TRANSPORTATION_DOMAIN_KEY, ALL_DOMAINS_KEY]
save_key = 'transport_vs_other'
df = to_df(stats_all, model_name, answer_type, prompt_type, subs, domains)

In [26]:
df

,TRANSPORTATION,NON_TRANSPORTATION,ALL_DOMAINS
"(1, object_tracking, R, T/F)",0.026827,0.036967,0.031835
"(1, object_tracking, No R, T/F)",0.022657,0.022094,0.022366
"(1, fluent_tracking, R, T/F)",0.193188,0.146300,0.169200
"(1, fluent_tracking, No R, T/F)",0.176088,0.137166,0.155340
"(1, state_tracking, R, T/F)",0.421429,0.402439,0.414414
...,...,...,...
"(19, numerical_reasoning, No R, T/F)",0.057857,0.032500,0.046154
"(19, hallucination, R, T/F)",0.022472,0.016414,0.019620
"(19, hallucination, No R, T/F)",0.040000,0.026923,0.033544
"(19, all_categories, R, T/F)",0.097064,0.078419,0.088308


In [31]:
latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)

latex_table_all = r"""
\begin{table*}[h!]
\begin{adjustbox}{width=2\columnwidth,center}
""" + latex_table + """
\end{adjustbox}
\caption{}
\end{table*}
"""
os.makedirs(os.path.join(STATISTICS_PATH, 'tables_2'), exist_ok=True)
with open(os.path.join(STATISTICS_PATH, 'tables_2', f'{model_name}.{prompt_type}.{answer_type_ext}.{save_key}.tex'), 'w') as f:
    f.write(latex_table_all)

/var/folders/_0/bqv6dr454lsgny6_gs_xb0q80000gn/T/ipykernel_61546/3966389069.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)


In [32]:
latex_table_all

'\n\\begin{table*}[h!]\n\\begin{adjustbox}{width=2\\columnwidth,center}\n\\begin{tabular}{lrrr}\n\\toprule\n{} &  TRANSPORTATION &  NON\\_TRANSPORTATION &  ALL\\_DOMAINS \\\\\n\\midrule\n(1, object\\_tracking, R, T/F)          &            0.03 &                0.04 &         0.03 \\\\\n(1, object\\_tracking, No R, T/F)       &            0.02 &                0.02 &         0.02 \\\\\n(1, fluent\\_tracking, R, T/F)          &            0.19 &                0.15 &         0.17 \\\\\n(1, fluent\\_tracking, No R, T/F)       &            0.18 &                0.14 &         0.16 \\\\\n(1, state\\_tracking, R, T/F)           &            0.42 &                0.40 &         0.41 \\\\\n(1, state\\_tracking, No R, T/F)        &            0.40 &                0.40 &         0.40 \\\\\n(1, action\\_executability, R, T/F)     &            0.18 &                0.09 &         0.14 \\\\\n(1, action\\_executability, No R, T/F)  &            0.16 &                0.08 &         0.12 \\\\\n(1, e